In [1]:
# 모델 구축
# 입력값 토큰화, 임베딩
# 훈련을 정의하고 학습
# 평가
# 예측하기 위한 데이터를 토큰화, 임베딩후 predict
# 라벨 출력

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [3]:
enc_IN = Input(shape=(4, 171)) # 영문 단어가 입력, 영문 단어는 4자짜리, 모든 문자의 수는 171개

# hidden state와 cell state를 전달해야 하므로 return_state=True 추가
_, state_h, state_c = LSTM(128, return_state=True)(enc_IN) 

# link = [state_h, state_c]
dec_IN = Input(shape=(3, 171)) # 한글 단어는 2자짜리+<Start>, 모든 문자의 수는 171개

# LSTM 모든 단의 출력을 사용해야 하므로 return_sequences=True 추가
dec_Y = LSTM(128, return_sequences=True)(dec_IN, initial_state=[state_h, state_c]) 

dec_OUT = Dense(171, activation='softmax')(dec_Y)
model = Model(inputs=[enc_IN, dec_IN], outputs=[dec_OUT])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 4, 171)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 3, 171)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 128), (None,      │         153,600 │ input_layer[0][0]          │
│                               │ 128), (None, 128)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ (None, 3, 128)            │         153,600 │ input_layer_1[0][0],       │
│                               │                           │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 3, 171)            │          22,059 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 329,259 (1.26 MB)

 Trainable params: 329,259 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam') # metrics가 없으면 훈련시 loss만 출력, predict()를 사용 못함

In [5]:
import pandas as pd
import numpy as np
# <START>, <END>, <PAD>
arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
arr2 = pd.read_csv('korean.csv', header=None)
# print(arr2[0].values.tolist())
num_to_char = arr1 + arr2[0].values.tolist()
print(len(num_to_char))

171


In [6]:
char_to_num = {char:i for i, char in enumerate(num_to_char)}
# print(char_to_num)

In [7]:
raw = pd.read_csv('translate.csv', header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [8]:
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng]
print(temp_eng_n)
temp_kor = '사랑'
np.eye(171)[temp_eng_n].shape

[14, 17, 24, 7]


(4, 171)

In [9]:
def encode(eng_kor):
    enc_in = []
    dec_in = []
    rnn_out = [] # decoder output
    for seq in eng_kor:
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(171)[eng])

        kor = [char_to_num[c] for c in ('S'+seq[1])]
        dec_in.append(np.eye(171)[kor])

        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    rnn_out = np.array(rnn_out)
    rnn_out = np.expand_dims(rnn_out, axis=2)
    return enc_in, dec_in, rnn_out

In [10]:
sample = [['word', '단어']]
encode(sample)[2]

array([[[ 61],
        [114],
        [  1]]])

In [11]:
X_enc, X_dec, y_rnn = encode(eng_kor)

In [12]:
%%time
model.fit([X_enc, X_dec], y_rnn, epochs=500)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 5.1332
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.0946 
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.0445 
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.9617 
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.8037 
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.4706 
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.8258 
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.4606 
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.6287 
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.4318 
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3622 
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.3930 
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.3279 
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.2950 
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.2791 
Epoch 16/500
4/4 ━━━

In [13]:
enc_in, dec_in, _ = encode([['tall', 'PP']])
# print(enc_in)
pred = model.predict([enc_in, dec_in])
# print(pred.shape)
word = np.argmax(pred[0], axis=-1)
# print(word)
num_to_char[word[0]], num_to_char[word[1]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step


('크', '다')

In [14]:
from numpy.random import randint
pick = randint(0, len(eng_kor), 5)
choose = [ [eng_kor[i][0], 'PP'] for i in pick]
print(choose)

[['cost', 'PP'], ['word', 'PP'], ['feet', 'PP'], ['meat', 'PP'], ['news', 'PP']]


In [15]:
enc_in, dec_in, _ = encode(choose)
pred = model.predict([enc_in, dec_in])
print(pred.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
(5, 3, 171)


In [16]:
for i in range(len(choose)):
    eng = choose[i][0]
    word = np.argmax(pred[i], axis=-1)
    kor = ''
    for j in range(2):
        kor = kor + num_to_char[word[j]]
    print(eng, kor)

cost 비용
word 단어
feet 다리
meat 고기
news 뉴스
